In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. EDA

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


In [ ]:
train = pd.read_csv("/kaggle/input/instant-gratification/train.csv")
test = pd.read_csv("/kaggle/input/instant-gratification/test.csv")
submission = pd.read_csv("/kaggle/input/instant-gratification/sample_submission.csv")

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
print(train["target"].value_counts())

타깃값의 불균형 문제 없음

In [ ]:
train["target"].cumsum().plot()

타깃값의 누출문제X

In [ ]:
train.isnull().sum().sum()

결측치X

In [ ]:
train.dtypes.value_counts()

In [ ]:
train.select_dtypes(include=["int64", "object"]).head()

In [ ]:
train_columns= [c for c in train.columns if c not in ["id", "target"]]
train[train_columns].nunique()
#id,target을 제외한 train의 unique한 행 개수

In [ ]:
print(np.where(train[train_columns].nunique()<250000))
#대부분의 열이 25만개이상의 유니크값 가짐 -> 25만개 이하의 유니크값 가지는 열 확인

In [ ]:
train_columns[146]

In [ ]:
print(train["wheezy-copper-turtle-magic"].nunique())
# 확연하게 작은 unique값 가지는 열

In [ ]:
train["wheezy-copper-turtle-magic"].value_counts().mean()
#해당열은 512개의 유니크한 값을 가지며 유니크한 값이 평균 512개의 개수를 가집

In [ ]:
train["wheezy-copper-turtle-magic"].hist()

In [ ]:
train_columns= [c for c in train_columns if c not in ["id", "target", "wheezy-copper-turtle-magic"]]

describe_train = train[train_columns].describe().T.drop("count",axis=1)
cmap = sns.diverging_palette(5,250,as_cmap=True)
describe_train.T.style.background_gradient(cmap,axis=1)
#-15~-20의 최솟값부터 +15~+20까지의  최댓값을 보통 가짐

In [ ]:
train_columns = [c for c in train.columns if c not in ["id", "target"]]
target_0_df = train.loc[train["target"]==0]
target_1_df = train.loc[train["target"]==1]

plt.figure(figsize=(24,24))
for index,name in enumerate(train_columns[:36]):
    plt.subplot(6,6,index+1)
    sns.histplot(target_0_df[name], color='blue', label='0', kde=False, stat="density", bins=30, alpha=0.5)
    sns.histplot(target_1_df[name], color='red', label='1', kde=False, stat="density", bins=30, alpha=0.5)
    plt.title(name)
    plt.xlabel("")
    plt.legend(["0","1"])
plt.tight_layout()
plt.show()

# 2. Modeling

## 2-1 1단게 모델링 (GMM , 6가지 모델 혼합)

"wheezy-copper-turtle-magic"를 범주형 변수로 인식하고 "wheezy-copper-turtle-magic"==0인 데이터를 기준으로
각열의 표준편차가 1 / 3~4인 데이터로 구분 됨

각각 두개의 모집단으로 구분하고 이를 이용해 GMM(Gaussian Mixture Model)사용 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn import svm
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import neighbors
from sklearn import linear_model
from tqdm.auto import tqdm
from sklearn.decomposition import KernelPCA
from sklearn.mixture import GaussianMixture as GMM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as  lgbm

In [ ]:
svnu_params = {
   "probability": True, 
    "kernel": "poly",
    "degree": 4,
    "gamma": "auto",
    "nu": 0.4,
    "coef0": 0.08,
    "random_state":4,
}
svnu2_params = {
   "probability": True, 
    "kernel": "poly",
    "degree": 2,
    "gamma": "auto",
    "nu": 0.4,
    "coef0": 0.08,
    "random_state":4,
}
qda_params = {"reg_param":0.111}
svc_params = {
    "probability": True,
    "kernel": "poly",
    "degree":4,
    "gamma":"auto",
    "random_state":4,
}
neighbor_params = {"n_neighbors":16}
lr_params = {
    "solver": "liblinear",
    "penalty":"l1",
    "C": 0.05,
    "random_state": 42}

In [ ]:
class hist_model(object):
    
    def __init__(self, bins=50):
        self.bins = bins
        
    def fit(self, X):
        bin_hight, bin_edge = [], []
        
        for var in X.T:
            # get bins hight and interval
            bh, bedge = np.histogram(var, bins=self.bins)
            bin_hight.append(bh)
            bin_edge.append(bedge)
        
        self.bin_hight = np.array(bin_hight)
        self.bin_edge = np.array(bin_edge)

    def predict(self, X):
        
        scores = []
        for obs in X:
            obs_score = []
            for i, var in enumerate(obs):
                # find wich bin obs is in
                bin_num = (var > self.bin_edge[i]).argmin()-1
                obs_score.append(self.bin_hight[i, bin_num]) # find bin hitght
            
            scores.append(np.mean(obs_score))
        
        return np.array(scores)

In [ ]:
def run_model(clf_list, train, test, random_state, gmm_init_params="kmeans"):
    #들어온 모델의 개수, 예측용 훈련/테스트셋 만들기 
    MODEL_COUNT = len(clf_list)
    oof_train = np.zeros((len(train), MODEL_COUNT))
    oof_test = np.zeros((len(test), MODEL_COUNT))
    
    #필요한 열만 선택(id, target, 범주형 변수 제거)
    train_columns = [c for c in train.columns if c not in ["id", "target","wheezy-copper-turtle-magic"]]
    
    #tqdm은 반복의 경과를 볼 수있는 라이브러리 ->훈련과정을 보여줌
    # 범주형 변수에 대하여 모두 탐색해보며 각각의 열의 분포 확인하기
    for i in tqdm(range(512)):
        x_train = train[train["wheezy-copper-turtle-magic"]==i]
        x_test = test[test["wheezy-copper-turtle-magic"]==i]
        print("i: ", i, x_train.shape, x_test.shape)
        
        train_idx_origin = x_train.index
        test_idx_origin = x_test.index
        
        train_std = x_train[train_columns].std()
        # -10~10의 분포 데이터 (-> 표준편차 3)와 -4~4 분포 데이터(-> 표준편차 1)로 구분되어짐
        #표준편차가 2보다 큰 열이 더 성능좋음
        cols = list(train_std.index.values[np.where(train_std > 2)])
        
        x_train = x_train.reset_index(drop=True)
        y_train = x_train.target
        
        x_train = x_train[cols].values
        x_test = x_test[cols].values
        
        all_data = np.vstack([x_train, x_test])
        
        #Kernel PCA
        all_data = KernelPCA(
            n_components=len(cols), kernel = "cosine", random_state=random_state).fit_transform(all_data)
        
        #GMM
        gmm = GMM(
            n_components=5,
            random_state=random_state,
            max_iter=1000,
            init_params=gmm_init_params,
        ).fit(all_data)
        
        gmm_pred = gmm.predict_proba(all_data)
        gmm_score = gmm.score_samples(all_data).reshape(-1,1)
        gmm_label = gmm.predict(all_data)
        
        #hist
        hist = hist_model()
        hist.fit(all_data)
        hist_pred = hist.predict(all_data).reshape(-1,1)
        
        all_data = np.hstack([all_data, gmm_pred, gmm_pred, gmm_pred, gmm_pred, gmm_pred])
        all_data= np.hstack([all_data, hist_pred, gmm_score, gmm_score, gmm_score])
        
        #ss
        all_data = StandardScaler().fit_transform(all_data)
        
        x_train = all_data[:x_train.shape[0]]
        x_test = all_data[x_train.shape[0]:]
        
        #KFold
        fold = StratifiedKFold(n_splits=5,shuffle=True, random_state=random_state)
        for trn_idx, val_idx in fold.split(x_train, gmm_label[:x_train.shape[0]]):
            for model_index, model in enumerate(model_list):
                model.fit(x_train[trn_idx], y_train[trn_idx])
        
                oof_train[train_idx_origin[val_idx], model_index] = model.predict_proba(x_train[val_idx])[:,1]
                if x_test.shape[0] == 0:
                    continue
                oof_test[test_idx_origin, model_index] += (model.predict_proba(x_test)[:,1] / fold.n_splits)
    
    for i, clf in enumerate(clf_list):
        print(clf)
        print(roc_auc_score(train['target'], oof_train[:, i]))
        print()
            
    oof_train_df = pd.DataFrame(oof_train)
    oof_test_df = pd.DataFrame(oof_test)
    
    return oof_train_df, oof_test_df


In [ ]:
nusvc_model = svm.NuSVC(**svnu_params)
nusvc2_model = svm.NuSVC(**svnu2_params)
qda_model = QuadraticDiscriminantAnalysis(**qda_params)
svc_model = svm.SVC(**svc_params)
knn_model = neighbors.KNeighborsClassifier(**neighbor_params)
lr_model = linear_model.LogisticRegression(**lr_params)

model_list = [nusvc_model, nusvc2_model, qda_model, svc_model, knn_model, lr_model]
oof_train_kmeans_seed1, oof_test_kmenas_seed1 = run_model(model_list, train, test,1)
oof_train_kmeans_seed2, oof_test_kmenas_seed2 = run_model(model_list, train, test,2)
oof_train_random_seed1, oof_test_random_seed1 = run_model(model_list, train, test,1, "random")
oof_train_random_seed2, oof_test_random_seed2 = run_model(model_list, train, test,2, "random")

In [1]:
#1단계 모델링 종료  2단계 모델링에 사용할 train/test만들기
train_second = (oof_train_kmeans_seed1 + oof_train_kmeans_seed2 + oof_train_random_seed1 + oof_train_random_seed2)/4
test_second = (oof_test_kmeans_seed1 + oof_test_kmeans_seed2 + oof_test_random_seed1 + oof_test_random_seed2)/4
print('Ensemble', roc_auc_score(train['target'], train_second.mean(1)))

NameError: name 'oof_train_kmeans_seed1' is not defined

## 2-2 2단계 모델링 (lgbm+MLP)

In [ ]:
SEED_NUMBER =4
NFOLD=5

y_train = train["target"]
oof_lgbm_meta_train - no.zeros((len(train_second), SEED_NUMBER))
oof_lgbm_meta_test - no.zeros((len(train_second), SEED_NUMBER))
oof_mlp_meta_train - no.zeros((len(train_second), SEED_NUMBER))
oof_mlp_meta_train - no.zeros((len(train_second), SEED_NUMBER))

for seed in range(SEED_NUMBER):
    print("SEED Ensemble:", seed)
    mp16_params["random_state"]=seed
    lgbm_meta_params["seed"]=seed
    folds=StratifiedKFold(n_splits=NFOLD, shuffle=true, random_state=seed)
    for fold_index, (trn_index, val_index) in enumerate(
        folds.split(train_second, y_train),1
    ):
        print(f"{fold_index} FOLD Start")
        trn_x, trn_y = train_second.iloc[trn_index], y_train.iloc[trn_index]
        val_x, val_y = train_second.iloc[val_index], y_train.iloc[val_index]
        
        #MLP
        mlp_meta_model = neural_network.MLPClassifier(**mlp16_parmas)
        mlp_meta_model.fit(trn_x, trn_y)
        
        oof_mlp_meta_train[val_index, seed] = mlp_meta_model.predict_proba(val_x)[:,1]
        oof_mlp_meta_test[:, seed] += (mlp_meta_model.predict_proba(test_second[:,1]) / NFOLD)
        print("MLP META SCORE: ", roc_auc_score(val_y,oof_mlp_meta_train[val_index, seed]),
             )
        
        #lgbm
        dtrain=lgbm.Dataset(trn_x, label=trn_y, silent=True)
        dcross=lgbm.Dataset(val_x, label=val_y, silent=True)
        
        lgbm_meta_model = lgbm.train(
            lgbm_meta_param,
            train_set=dtrain,
            valid_sets=[dtrain, dcross],
            verbose_eval=False,
            early_stopping_rounds=100,
        )
        oof_lgbm_meta_train[val_index, seed] = lgbm_meta_model.predict(val_x)
        oof_lgbm_meta_test[:, seed] += (mlp_meta_model.predict(test_second) / NFOLD)
        print("LGBM META SCORE: ", roc_auc_score(val_y, oof_lgbm_meta_train[val_index, seed]), )
        

In [ ]:
oof_lgbm_meta_train_df = (
    pd.DataFrame(oof_lgbm_meta_train).mean(axis=1).to_frame().rename(columns={0: "lgbm"})
    )
oof_lgbm_meta_test_df=(pd.DAtaFrame(oof_lgbm_meta_test).mean(axis=1).to_frame().rename(columns={0: "lgbm"})
oof_mlp_meta_train_df=(pd.DAtaFrame(oof_mlp_meta_train).mean(axis=1).to_frame().rename(columns={0: "mlp"})
oof_mlp_meta_test_df=(pd.DAtaFrame(oof_mlp_meta_test).mean(axis=1).to_frame().rename(columns={0: "mlp"})

oof_train_third = pd.concat([train_second, oof_lgbm_meta_train_df, oof_mlp_meta_train_df], axis=1)
oof_test_third = pd.concat([test_second, oof_lgbm_meta_test_df, oof_mlp_meta_test_df], axis=1)
                      
print("Ensemble", roc_auc_score(train["target"], oof_train_third.mean(1)))
                     